In [1]:
import requests
from time import sleep
from ipyleaflet import Map, Marker, Rectangle
from ipywidgets import HTML, VBox, IntProgress
import threading

# Créer une carte centrée sur Bordeaux
center = [44.8378, -0.5792]
m = Map(center=center, zoom=13)

# Conteneur pour afficher les coordonnées sélectionnées et les adresses
output = HTML("<strong>Coordonnées sélectionnées :</strong><br>Aucune sélection pour l'instant.")
address_output = HTML("<strong>Adresses trouvées :</strong><br>Aucune adresse pour l'instant.")
progress_bar = IntProgress(min=0, max=1, value=0, description='Progression')  # Barre de progression
display(VBox([m, output, address_output, progress_bar]))

# Variables pour stocker les deux points cliqués
start_point = None
end_point = None
rectangle = None
markers = []  # Liste pour stocker les marqueurs ajoutés

def handle_map_click(**kwargs):
    global start_point, end_point, rectangle, markers
    
    if kwargs.get('type') == 'click':
        latlng = kwargs.get('coordinates')
        
        # Si deux points ont déjà été sélectionnés, on réinitialise tout
        if start_point is not None and end_point is not None:
            # Supprimer l'ancien rectangle et les marqueurs de la carte
            if rectangle is not None:
                m.remove_layer(rectangle)
            for marker in markers:
                m.remove_layer(marker)
            markers.clear()
            
            # Réinitialiser les points
            start_point = None
            end_point = None
            rectangle = None
            address_output.value = "<strong>Adresses trouvées :</strong><br>Aucune adresse pour l'instant."
            progress_bar.value = 0
        
        # Enregistrer le premier ou le deuxième point
        if start_point is None:
            start_point = latlng
            marker = Marker(location=start_point)
            markers.append(marker)
            m.add_layer(marker)
            output.value = f"Point 1 sélectionné : {start_point}"
        
        elif end_point is None:
            end_point = latlng
            marker = Marker(location=end_point)
            markers.append(marker)
            m.add_layer(marker)
            
            # Dessiner un rectangle entre les deux points
            bounds = [start_point, end_point]
            rectangle = Rectangle(bounds=bounds, color="blue", fill_opacity=0.3)
            m.add_layer(rectangle)
            
            output.value = (f"<strong>Coordonnées sélectionnées :</strong><br>"
                            f"Coin supérieur gauche : {start_point}<br>"
                            f"Coin inférieur droit : {end_point}")

            # Exécuter la récupération des adresses dans un thread séparé
            threading.Thread(target=get_addresses_with_overpass, args=(start_point, end_point)).start()

def get_addresses_with_overpass(start, end):
    """
    Récupère les objets avec des adresses dans la zone définie
    par deux points GPS en utilisant l'API Overpass.
    
    :param start: Coordonnées du coin supérieur gauche (lat, lon)
    :param end: Coordonnées du coin inférieur droit (lat, lon)
    """
    
    overpass_url = "http://overpass-api.de/api/interpreter"
    
    # Extraire les coordonnées des points
    lat1, lon1 = start  # Point 1 (coin supérieur gauche)
    lat2, lon2 = end    # Point 2 (coin inférieur droit)
    
    # Construire la bounding box dans le bon ordre : south, west, north, east
    bbox = f"{min(lat1, lat2)},{min(lon1, lon2)},{max(lat1, lat2)},{max(lon1, lon2)}"
    
    # Construire la requête Overpass
    query = f"""
    [out:json];
    (
      node["addr:housenumber"]({bbox});
      way["addr:housenumber"]({bbox});
      relation["addr:housenumber"]({bbox});
    );
    out body;
    """

    # Afficher l'URL complète de la requête dans la sortie
    print(f"URL de la requête : {overpass_url}?data={query}")
    
    try:
        response = requests.get(overpass_url, data=query, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        if data['elements']:
            addresses = []
            total_elements = len(data['elements'])
            progress_bar.max = total_elements  # Définir la taille maximale de la barre de progression
            
            for i, element in enumerate(data['elements'], start=1):
                tags = element.get('tags', {})
                housenumber = tags.get('addr:housenumber', '')
                street = tags.get('addr:street', '')
                city = tags.get('addr:city', '')
                postcode = tags.get('addr:postcode', '')
                
                # Vérifier si l'élément est résidentiel
                if not tags.get('name') and not tags.get('shop'):
                    residential_status = " (résidentiel)"
                else:
                    residential_status = " (non résidentiel)"
                
                # Si le nom de la rue est manquant, utiliser le reverse geocoding
                if housenumber or not street:
                    lat = element['lat']
                    lon = element['lon']
                    reverse_address = reverse_geocode(lat, lon)
                    street = reverse_address.get("street", "")
                    city = reverse_address.get("city", "")
                
                # Construire l'adresse complète
                address_parts = [housenumber, street, city, postcode]
                address = ", ".join(part for part in address_parts if part) + residential_status
                
                if not address.strip():  # Si aucune info d'adresse n'est disponible, utiliser lat/lon
                    lat = element['lat']
                    lon = element['lon']
                    address = f"Adresse inconnue (lat={lat}, lon={lon})"
                
                addresses.append(address)
                
                # Mettre à jour la barre de progression
                progress_bar.value = i
                progress_bar.description = f"{i}/{total_elements}"
                sleep(0.1)  # Simuler un délai pour mieux visualiser la progression

            address_output.value = ("<strong>Adresses trouvées :</strong><br>" +
                                    "<br>".join(addresses))
        else:
            address_output.value = "<strong>Adresses trouvées :</strong><br>Aucune adresse trouvée."
            progress_bar.value = 0
    
    except requests.exceptions.RequestException as e:
        address_output.value = f"<strong>Erreur lors de la récupération des adresses :</strong><br>{e}"
        progress_bar.value = 0
        print(f"Erreur : {e}")
        
def reverse_geocode(lat, lon):
    """
    Utilise l'API Nominatim pour effectuer un reverse geocoding
    et récupérer l'adresse complète à partir des coordonnées GPS.
    
    :param lat: Latitude
    :param lon: Longitude
    :return: Dictionnaire contenant 'street' et 'city'
    """
    nominatim_url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": lat,
        "lon": lon,
        "format": "json",
        "addressdetails": 1
    }
    
    headers = {
        "User-Agent": "MyPythonApp/1.0 (contact@example.com)"
    }
    
    try:
        response = requests.get(nominatim_url, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        address = data.get("address", {})
        return {
            "street": address.get("road", ""),
            "city": address.get("city", "")
        }
    
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du reverse geocoding : {e}")
        return {"street": "", "city": ""}


# Lier la fonction de gestion des clics à la carte
m.on_interaction(handle_map_click)

URL de la requête : http://overpass-api.de/api/interpreter?data=
    [out:json];
    (
      node["addr:housenumber"](44.83752593804117,-0.578632950782776,44.83782600598757,-0.5769914388656617);
      way["addr:housenumber"](44.83752593804117,-0.578632950782776,44.83782600598757,-0.5769914388656617);
      relation["addr:housenumber"](44.83752593804117,-0.578632950782776,44.83782600598757,-0.5769914388656617);
    );
    out body;
    
URL de la requête : http://overpass-api.de/api/interpreter?data=
    [out:json];
    (
      node["addr:housenumber"](44.83741384795199,-0.57878315448761,44.83792687374903,-0.5764630436897279);
      way["addr:housenumber"](44.83741384795199,-0.57878315448761,44.83792687374903,-0.5764630436897279);
      relation["addr:housenumber"](44.83741384795199,-0.57878315448761,44.83792687374903,-0.5764630436897279);
    );
    out body;
    
URL de la requête : http://overpass-api.de/api/interpreter?data=
    [out:json];
    (
      node["addr:housenumber"](44.83